# Класифікація відео на основі класифікатору зображень OpenAI CLIP

Встановлюємо Conda для Google Colab

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


Імпортимо та дивимося чи все гаразд

In [2]:
import condacolab
condacolab.check()
!conda --version

✨🍰✨ Everything looks OK!
conda 23.11.0


Встановлюємо необхідні модулі

In [3]:
!conda install --yes -c pytorch pytorch torchvision cudatoolkit
!pip install opencv-python
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install yt-dlp

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    blas-2.116                 |              mkl          13 KB  conda-forge
    blas-devel-3.9.0           |   16_linux64_mkl          12 KB  conda-forge
    ca-certificates-2024.7.4   |       hbcca054_0         151 KB  conda-forge
    certifi-2024.7.4           |     pyhd8ed1ab_0         156 KB  conda-forge
    cudatoolkit-11.8.0         |    

Викачуємо рекламне відео від NASA для класифікації

In [4]:
%%shell

VIDEO_URL_PATH="https://www.youtube.com/watch?v=yVcxTCNsFHQ"
VIDEO_SAVE_NAME="nasa"
FORMAT_OPTIONS="bestvideo[height<=720][ext=mp4][vcodec^=avc]+bestaudio[ext=m4a]/best[ext=mp4]/best"


yt-dlp -f "$FORMAT_OPTIONS" "$VIDEO_URL_PATH" --merge-output-format mp4 -o "$VIDEO_SAVE_NAME"

[youtube] Extracting URL: https://www.youtube.com/watch?v=yVcxTCNsFHQ
[youtube] yVcxTCNsFHQ: Downloading webpage
[youtube] yVcxTCNsFHQ: Downloading ios player API JSON
[youtube] yVcxTCNsFHQ: Downloading web creator player API JSON
[youtube] yVcxTCNsFHQ: Downloading m3u8 information
[info] yVcxTCNsFHQ: Downloading 1 format(s): 136+140
[download] nasa.mp4 has already been downloaded


Викачуємо відео з собакою для класифікації

In [5]:
%%shell

VIDEO_URL_PATH="https://videos.pexels.com/video-files/3042473/3042473-uhd_2560_1440_30fps.mp4"
VIDEO_PATH="/content/silly_dog.mp4"

if [ -f "$VIDEO_PATH" ]
then
    echo "Video already installed at $VIDEO_PATH"
else
    wget -cO - "$VIDEO_URL_PATH" > "$VIDEO_PATH"
fi


Video already installed at /content/silly_dog.mp4


Викачуємо відео з котом для класифікації

In [6]:
%%shell

VIDEO_URL_PATH="https://videos.pexels.com/video-files/855282/855282-hd_1280_720_25fps.mp4"
VIDEO_PATH="/content/silly_cat.mp4"

if [ -f "$VIDEO_PATH" ]
then
    echo "Video already installed at $VIDEO_PATH"
else
    wget -cO - "$VIDEO_URL_PATH" > "$VIDEO_PATH"
fi

Video already installed at /content/silly_cat.mp4


Викачуємо тестове зображення з котом, що стоїть на аватарці CS50

In [7]:
%%shell

IMAGE_PATH="/content/cs50_cat.jpg"
IMAGE_URL="https://i.chzbgr.com/full/875511040/h8EB4D6E9/famous-cat-meme-which-started-and-launched-the-website-i-can-haz-cheezburger"

if [ -f "$IMAGE_PATH" ]
then
    echo "Image already installed at $IMAGE_PATH"
else
    wget -cO - "$IMAGE_URL" > "$IMAGE_PATH"
fi

Image already installed at /content/cs50_cat.jpg


Імпортуємо необхідні бібліотеки

In [8]:
import cv2
import numpy as np
import torch
import clip
from PIL import Image

Задаємо seed для PyTorch та NymPy

In [10]:
torch.manual_seed(17)
np.random.seed(17)

Дивимося доступні моделі

In [11]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

Протестуємо CLIP для тестового зображення

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("cs50_cat.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a blueprint", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    print(image_features.shape)
    text_features = model.encode_text(text)
    print(text_features.shape)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)
print("Label probs:", probs)

torch.Size([1, 512])
torch.Size([3, 512])
Label probs: [[0.000785 0.038185 0.96103 ]]


Робимо клас для класифікатору вмісту відео ряду на основі моделей класифікації зображень OpenAI CLIP.

### Принцип роботи:

На першій ітерації з відео ряду вибираються рівновіддалені кадри з кроком `frame_step`, починаючи з `frame_step`-го кадру. У другому циклі також беруться рівновіддалені кадри з кроком `frame_step`, проте першим кадром у послідовності буде `frame_step / 2`-й. На третій ітерації `frame_step / 4`-й. На четвертій `frame_step / 8`-й. І так допоки початковий елемент не буде нульовим (не включаючи), або допоки різниця між ймовірностями у попередньому та поточному кроці не буде менше за певний допуск $𝜺$. Різниця між векторами ймовірностей рахується як норма різниці векторів.

In [16]:
class VideoClassifier:
    """Video classifier class. Classifies video using `classify_video` method.
    """
    def __init__(
            self,
            model_name="ViT-B/32",
            device="cpu",
            frame_step_relative=0.15,
            eps=1e-3
    ):
        """
        Inputs:
            model_name:             str   -- model name from `clip.available_models()`.
            device:                 str   -- either 'cpu' or 'cuda', others are not supported.
            frame_step_relative:    float -- proportion of frame step to the number of frames, should
                                             be between 0 and 0.5 exluding.
            eps:                    float -- precision up to which probabilites won't be updated
        """
        if model_name not in clip.available_models():
            raise ValueError(f"{model_name} is not in available models")
        if device not in ["cpu", "cuda"]:
            raise ValueError(f"Unsupported device {device}")
        self.device = device
        if 0.0 > frame_step_relative > 0.5:
            raise ValueError(f"Wrong frame step: {frame_step_relative * 100}%")
        self.frame_step_relative = frame_step_relative
        self.eps = eps
        self.model, self.modelpreprocess = clip.load(model_name, device=self.device)

    def _classify_image_(self, image, labels):
        """
        Softly classifies the input `image` of either `Numpy.Ndarray` or `PIL.Image` types
        according to the `labels` list of strings.
        """
        image_preprocessed = preprocess(image).unsqueeze(0).to(device)
        text = clip.tokenize(labels).to(device)

        with torch.no_grad():
            logits_per_image, logits_per_text = model(image_preprocessed, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

        return probs

    def classify_video(self, frames_list, labels):
        """
        Classifies the video sequence using frames from `frame_list`. `frame_list` can contain either
        `Numpy.Ndarray` type or `PIL.Image` objects. `labels` contains text representation of objects
        we are looking for.
        """
        frame_step = int(self.frame_step_relative * len(frames_list))
        init_frame = frame_step
        current_probs = np.zeros((1, len(labels)))
        frames_counter = 0
        prev_probs = None

        while init_frame != 0 or np.linalg.norm(current_probs - prev_probs) > self.eps:

            # Copy current probabilities to the previous and convert current to sum instead of mean
            prev_probs = current_probs.copy()
            current_probs *= frames_counter

            for frame_index in np.arange(init_frame, len(frames_list), frame_step):
                current_probs += self._classify_image_(frames_list[frame_index], labels)
                frames_counter += 1

            # Revert updated sum to updated mean
            current_probs /= frames_counter
            init_frame //= 2

        return current_probs


Створюємо функцію, що зчитує відео та повертає список кадрів.

In [14]:
def video2frames(video_path, interval):
  """Returns a list of frames in PIL
  """
  video_frames = []
  video_cap = cv2.VideoCapture(video_path)
  ctr = 0

  while(video_cap.isOpened()):
    frame_captured, cv2_img = video_cap.read()
    if frame_captured and ctr % interval == 0:
      pil_img = Image.fromarray(cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB))
      video_frames.append(pil_img)
    #   video_frames.append(cv2_img)
    elif not frame_captured:
      video_cap.release()
  return video_frames

Верифікуємо класифікатор на скачаних відео.

In [ ]:
video_files = ["silly_cat.mp4", "silly_dog.mp4", "nasa.mp4"]
labels = ["a space", "a dog", "a cat"]
video_classifier = VideoClassifier("ViT-B/32", device, frame_step_relative=0.2)

for video_file in video_files:
    print(video_file)
    frames = video2frames(video_file, 1)
    probs = video_classifier.classify_video(frames, labels)
    for prob, label in zip(probs[0], labels):
        print(f"{label}: {prob}")
    print()
    del frames

Як бачимо, переважна маса ймовірностей зосереджена у потрібних позначках. Отже, класифікатор змісту відео працює справно!